In [2]:
# reload imports when code changes
%load_ext autoreload
QEMU_BIN = "../../qemu/build/qemu-x86_64"
QEMU_PATCHED_BIN = "./qemu/build/qemu-x86_64"

from assemble import assemble
from utils import exec_cmd
import json

In [3]:
def get_qemu_cmd(ex):
    return QEMU_BIN + " -plugin ./inscount.so " + ex

def get_qemu_cmd_patched(ex):
    return QEMU_PATCHED_BIN + " " + ex

def bind_eff(cmd):
    eff_cores = exec_cmd("cat", ["/sys/devices/cpu_core/cpus"])
    if eff_cores.returncode != 0:
        print("Warning: I only have Intel computers so I don't know how to get the effective cores on your system")
        return cmd
    return "taskset -c " + eff_cores.stdout.strip() + " " + cmd


def run(cmd_func, nbNops = 100, nbRuns = 1000):
    ex = assemble("nop\n" * nbNops, "test")
    cmd = cmd_func(ex)
    cmd = "perf stat -j -e instructions,cycles,task-clock -r " + str(nbRuns) + " " + cmd
    cmd = bind_eff(cmd)
    print(cmd)
    cmd = cmd.split(" ")
    res = exec_cmd(cmd[0], cmd[1:])
    if res.returncode != 0:
        print("Error: " + res.stderr)
        return
    data = []
    for line in res.stderr.split("\n"):
        if len(line) == 0:
            continue
        data.append(json.loads(line))
    return (data, res.stdout, res.stderr)

In [4]:
def run_save(name, cmd_func, nbNops = 100, nbRuns = 1000):
    data, stdout, stderr = run(cmd_func, nbNops, nbRuns)
    with open(name + ".json", "w") as f:
        for d in data:
            f.write(json.dumps(d) + "\n")

In [6]:
for i in range(0, 10001, 1000):
    run_save("qemu_" + str(i), get_qemu_cmd, i, 5000)
    run_save("qemu_patched_" + str(i), get_qemu_cmd_patched, i, 5000)

taskset -c 0-11 perf stat -j -e instructions,cycles,task-clock -r 5000 ../../qemu/build/qemu-x86_64 -plugin ./inscount.so test.out
taskset -c 0-11 perf stat -j -e instructions,cycles,task-clock -r 5000 ./qemu/build/qemu-x86_64 test.out
taskset -c 0-11 perf stat -j -e instructions,cycles,task-clock -r 5000 ../../qemu/build/qemu-x86_64 -plugin ./inscount.so test.out
taskset -c 0-11 perf stat -j -e instructions,cycles,task-clock -r 5000 ./qemu/build/qemu-x86_64 test.out
taskset -c 0-11 perf stat -j -e instructions,cycles,task-clock -r 5000 ../../qemu/build/qemu-x86_64 -plugin ./inscount.so test.out
taskset -c 0-11 perf stat -j -e instructions,cycles,task-clock -r 5000 ./qemu/build/qemu-x86_64 test.out
taskset -c 0-11 perf stat -j -e instructions,cycles,task-clock -r 5000 ../../qemu/build/qemu-x86_64 -plugin ./inscount.so test.out
taskset -c 0-11 perf stat -j -e instructions,cycles,task-clock -r 5000 ./qemu/build/qemu-x86_64 test.out
taskset -c 0-11 perf stat -j -e instructions,cycles,task

In [23]:
def get_qemu_cmd_tblog(ex):
    return QEMU_BIN + " -plugin ./tblog.so " + ex

# Save also tb nb and sizes
def run_tblog(nbNops = 100):
    ex = assemble("nop\n" * nbNops, "test")
    cmd = get_qemu_cmd_tblog(ex)
    cmd = cmd.split(" ")
    res = exec_cmd(cmd[0], cmd[1:])
    if res.returncode != 0:
        print("Error: " + res.stderr)
        return
    data = []
    print(res)
    for line in res.stdout.split("\n"):
        if len(line) == 0:
            continue
        data.append(json.loads(line))
    return (data, res.stdout, res.stderr)

In [24]:
def run_tblog_save(name, nbNops = 100):
    data, stdout, stderr = run_tblog(nbNops)
    with open(name + ".json", "w") as f:
        for d in data:
            f.write(json.dumps(d) + "\n")

for i in range(0, 10001, 1000):
    run_tblog_save("qemu_tblog_" + str(i), i)

CompletedProcess(args=['../../qemu/build/qemu-x86_64', '-plugin', './tblog.so', 'test.out'], returncode=0, stdout='[{"type": "trans", "n_insn": 3},{"type": "exec", "n_insn": 3},{}]\n', stderr='')
CompletedProcess(args=['../../qemu/build/qemu-x86_64', '-plugin', './tblog.so', 'test.out'], returncode=0, stdout='[{"type": "trans", "n_insn": 512},{"type": "exec", "n_insn": 512},{"type": "trans", "n_insn": 491},{"type": "exec", "n_insn": 491},{}]\n', stderr='')
CompletedProcess(args=['../../qemu/build/qemu-x86_64', '-plugin', './tblog.so', 'test.out'], returncode=0, stdout='[{"type": "trans", "n_insn": 512},{"type": "exec", "n_insn": 512},{"type": "trans", "n_insn": 512},{"type": "exec", "n_insn": 512},{"type": "trans", "n_insn": 512},{"type": "exec", "n_insn": 512},{"type": "trans", "n_insn": 467},{"type": "exec", "n_insn": 467},{}]\n', stderr='')
CompletedProcess(args=['../../qemu/build/qemu-x86_64', '-plugin', './tblog.so', 'test.out'], returncode=0, stdout='[{"type": "trans", "n_insn": 